<a href="https://colab.research.google.com/github/sash-ko/ml-playground/blob/hands-on-m/chapter-10/MNIST_fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
tf.__version__

'2.2.0'

In [ ]:
keras.__version__